In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import os
from moscot.problems.time._lineage import TemporalProblem
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sc.settings.verbosity = 0

In [ ]:
sns.set_style('whitegrid')

sc.set_figure_params(dpi=200)

In [ ]:
palette=sns.color_palette()

In [ ]:
Path="/home/mgander/mouse_atlas/data/WOT"

In [ ]:
adata=sc.read(f"{Path}/reprogramming_schiebinger_serum.h5ad")
adata.obs['day']=adata.obs['day'].astype('float')
adata=adata[adata.obs['day']>7.9].copy()
adata.obs['day']=adata.obs['day'].astype('category')
days=sorted(set(adata.obs['day']))
# Every 0.5 days for 18 days 

In [ ]:
## Compute the mappings

In [14]:
eps=0.005
lam2=100
tau2=lam2/(lam2+eps)
tau1=0.98


def get_dfn(i):    
    
    day0=days[i]
    day1=days[i+1]

    print((day0,day1))
    adata_sub=sc.read(f"{Path}/anndatas/{day0}_{day1}_adata.h5ad")

    tp=TemporalProblem(adata_sub)
    #tp.score_genes_for_marginals(gene_set_proliferation='mouse',  gene_set_apoptosis='mouse')
    tp = tp.prepare('day', joint_attr=f'X_pca')
    
    result=tp.solve(epsilon=eps, tau_a=tau1, tau_b=tau2)
    
    gr=tp[(day0, day1)].solution.a
    gr=gr/gr.mean()
    
    adata2=adata_sub.copy()
    A0=adata_sub[adata_sub.obs['day']==day0].copy()
    adata2.obs['day']=adata2.obs['day'].astype('category')
    tp2=TemporalProblem(adata2)
    tp2.score_genes_for_marginals(gene_set_proliferation='mouse',  gene_set_apoptosis='mouse')
    tp2 = tp2.prepare('day')
    cc_scores=tp2.prior_growth_rates.loc[A0.obs.index]['prior_growth_rates']
    
    # Since we can only learn relative proliferation scores, normalize them to 1 
    cc_scores=cc_scores/np.mean(cc_scores)

    df=pd.DataFrame({'CellId':list(A0.obs.index), 'moscot_growth_rates':gr, 'scores':cc_scores})
    df=df.join(A0.obs).copy()
    return(df)

In [15]:
for i in range(len(days)-1):
    ts0=days[i]
    df=get_dfn(i)
    df.to_pickle(f'{Path}/growth_rates/moscot/{ts0}.pkl')

(8.0, 8.25)
INFO     Ordering Index(['D8_Dox_C1_AAACCTGAGCCGGTAA-1', 'D8_Dox_C1_AAACCTGCAGCCACCA-1',                           
                'D8_Dox_C1_AAACCTGGTGCCTTGG-1', 'D8_Dox_C1_AAACCTGGTTCCGTCT-1',                                    
                'D8_Dox_C1_AAACGGGAGCTGAACG-1', 'D8_Dox_C1_AAACGGGGTGATGATA-1',                                    
                'D8_Dox_C1_AAACGGGGTGCAGACA-1', 'D8_Dox_C1_AAACGGGGTGTCGCTG-1',                                    
                'D8_Dox_C1_AAACGGGTCTACTTAC-1', 'D8_Dox_C1_AAACGGGTCTGCAAGT-1',                                    
                ...                                                                                                
                'D8.25_serum_C2_TTTGCGCGTGTCAATC-1',                                                               
                'D8.25_serum_C2_TTTGCGCGTTCGCTAA-1',                                                               
                'D8.25_serum_C2_TTTGCGCTCATCATTC-1',        

CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12020, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


INFO     Ordering Index(['D8_Dox_C1_AAACCTGAGCCGGTAA-1', 'D8_Dox_C1_AAACCTGCAGCCACCA-1',                           
                'D8_Dox_C1_AAACCTGGTGCCTTGG-1', 'D8_Dox_C1_AAACCTGGTTCCGTCT-1',                                    
                'D8_Dox_C1_AAACGGGAGCTGAACG-1', 'D8_Dox_C1_AAACGGGGTGATGATA-1',                                    
                'D8_Dox_C1_AAACGGGGTGCAGACA-1', 'D8_Dox_C1_AAACGGGGTGTCGCTG-1',                                    
                'D8_Dox_C1_AAACGGGTCTACTTAC-1', 'D8_Dox_C1_AAACGGGTCTGCAAGT-1',                                    
                ...                                                                                                
                'D8.25_serum_C2_TTTGCGCGTGTCAATC-1',                                                               
                'D8.25_serum_C2_TTTGCGCGTTCGCTAA-1',                                                               
                'D8.25_serum_C2_TTTGCGCTCATCATTC-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(3573, 3088)].                                  
INFO     Ordering Index(['D8.5_serum_C1_AAACCTGCAAGACGTG-1', 'D8.5_serum_C1_AAACCTGCAGTGGGAT-1',                   
                'D8.5_serum_C1_AAACCTGCATGTCCTC-1', 'D8.5_serum_C1_AAACCTGTCCTAAGTG-1',                            
                'D8.5_serum_C1_AAACGGGAGCAAATCA-1', 'D8.5_serum_C1_AAACGGGAGGCGTACA-1',                            
                'D8.5_serum_C1_AAACGGGCATGCCTTC-1', 'D8.5_serum_C1_AAACGGGGTCTAAACC-1',                            
                'D8.5_serum_C1_AAAGATGGTACAAGTA-1', 'D8.5_serum_C1_AAAGATGGTCAAAGAT-1',                            
                ...                                                                                                
                'D8.75_serum_C2_TTTGCGCGTTGTGGAG-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(2982, 2266)].                                  
INFO     Ordering Index(['D9_serum_C1_AAACCTGGTGGTGTAG-1', 'D9_serum_C1_AAACCTGTCCAGGGCT-1',                       
                'D9_serum_C1_AAACGGGGTAACGACG-1', 'D9_serum_C1_AAACGGGGTCCAAGTT-1',                                
                'D9_serum_C1_AAAGCAAAGCTTTGGT-1', 'D9_serum_C1_AAAGCAAAGTCTTGCA-1',                                
                'D9_serum_C1_AAAGCAAAGTGGTAGC-1', 'D9_serum_C1_AAAGCAACAACACGCC-1',                                
                'D9_serum_C1_AAAGCAAGTAAGTGTA-1', 'D9_serum_C1_AAAGCAATCGAACTGT-1',                                
                ...                                                                                                
                'D9.5_serum_C2_TTTACTGCACTTAAGC-1', 'D9.5_serum_C2_TTTAC

...

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(2145, 2034)].                                  
INFO     Ordering Index(['D13_serum_C1_AAACCTGGTAATAGCA-1', 'D13_serum_C1_AAACCTGTCAACGGGA-1',                     
                'D13_serum_C1_AAACCTGTCGCCAGCA-1', 'D13_serum_C1_AAAGTAGAGTAAGTAC-1',                              
                'D13_serum_C1_AAATGCCGTCCGAAGA-1', 'D13_serum_C1_AACACGTAGGGCTTCC-1',                              
                'D13_serum_C1_AACACGTCATGCGCAC-1', 'D13_serum_C1_AACACGTGTGTGTGCC-1',                              
                'D13_serum_C1_AACCATGTCTACGAGT-1', 'D13_serum_C1_AACCGCGGTTAAGATG-1',                              
                ...                                                                                                
                'D13.5_serum_C2_TTTCCTCAGCGTTTAC-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(3758, 2723)].                                  
INFO     Ordering Index(['D14_serum_C1_AAACCTGAGAGGGATA-1', 'D14_serum_C1_AAACCTGCACGTAAGG-1',                     
                'D14_serum_C1_AAACCTGTCCTTTCGG-1', 'D14_serum_C1_AAACCTGTCGTGACAT-1',                              
                'D14_serum_C1_AAACGGGAGCTGTTCA-1', 'D14_serum_C1_AAACGGGAGGTAAACT-1',                              
                'D14_serum_C1_AAACGGGGTAGCGCAA-1', 'D14_serum_C1_AAACGGGGTATATGGA-1',                              
                'D14_serum_C1_AAACGGGGTCTCGTTC-1', 'D14_serum_C1_AAACGGGGTGGTCCGT-1',                              
                ...                                                                                                
                'D14.5_serum_C2_TTTGCGCAGCCCGAAA-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(3717, 4851)].                                  
INFO     Ordering Index(['D15_serum_C1_AAACCTGAGTGAACGC-1', 'D15_serum_C1_AAACCTGCACGAAATA-1',                     
                'D15_serum_C1_AAACCTGTCATCTGTT-1', 'D15_serum_C1_AAACCTGTCGGACAAG-1',                              
                'D15_serum_C1_AAACCTGTCGGCGCAT-1', 'D15_serum_C1_AAACCTGTCTGTGCAA-1',                              
                'D15_serum_C1_AAACGGGAGAGTACCG-1', 'D15_serum_C1_AAACGGGAGCTGCCCA-1',                              
                'D15_serum_C1_AAACGGGAGGCCATAG-1', 'D15_serum_C1_AAAGATGGTGCGGTAA-1',                              
                ...                                                                                                
                'D15.5_serum_C2_TTTGCGCCATTCACTT-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(3422, 4645)].                                  
INFO     Ordering Index(['D16_serum_C1_AAACCTGAGTCACGCC-1', 'D16_serum_C1_AAACCTGCATATACGC-1',                     
                'D16_serum_C1_AAACCTGGTCTAGCGC-1', 'D16_serum_C1_AAACCTGTCCCGACTT-1',                              
                'D16_serum_C1_AAACGGGCAGTGGAGT-1', 'D16_serum_C1_AAACGGGTCGGTGTTA-1',                              
                'D16_serum_C1_AAAGCAAGTCTGATTG-1', 'D16_serum_C1_AAATGCCAGCGATAGC-1',                              
                'D16_serum_C1_AAATGCCTCTCTTATG-1', 'D16_serum_C1_AACACGTAGTGTTTGC-1',                              
                ...                                                                                                
                'D16.5_serum_C2_TTTGCGCTCTGCTGCT-1',                    

INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(3678, 4068)].                                  
INFO     Ordering Index(['D17_serum_C1_AAACGGGCAACGCACC-1', 'D17_serum_C1_AAACGGGTCAAGGCTT-1',                     
                'D17_serum_C1_AAAGATGAGGTGTTAA-1', 'D17_serum_C1_AAAGATGAGTCGTTTG-1',                              
                'D17_serum_C1_AAAGATGCACTCAGGC-1', 'D17_serum_C1_AAAGCAACACTCAGGC-1',                              
                'D17_serum_C1_AAAGCAAGTTGATTCG-1', 'D17_serum_C1_AAAGTAGAGGGTTCCC-1',                              
                'D17_serum_C1_AAAGTAGCAAGTTGTC-1', 'D17_serum_C1_AAAGTAGCACAGTCGC-1',                              
                ...                                                                                                
                'D17.5_serum_C2_TTTGCGCCAATGCCAT-1',                    